In [4]:
using LinearAlgebra
using Random
using Flux
using Distributions

In [5]:
mutable struct RNNwaveFn
    Ns::Int #system size
    seed
    N_hid::Int # width of the hidden state 
    N_st::Int # number of states in sigle site
    model

    Nb::Int # number of sample
    sample::Matrix{Int}
    s0::Matrix{Float32}
end


In [3]:
function set_RNN(Ns0::Int, s0::Int, N_hid0::Int, N_st0::Int, Nb0::Int)
    ss = MersenneTwister(s0)
    model = Chain(LSTM(N_st0 => N_hid0; init=Flux.glorot_normal(s0)),Dense(N_hid0 => N_st0; init=Flux.randn32(ss)), softmax)
    sample = rand(ss, 1:N_st0, Nb0, Ns0)
    s0 = zeros(Float32, Ns0, N_st0)

    return Ns0, ss, N_hid0, N_st0, model, Nb0, sample
end


set_RNN (generic function with 1 method)

In [ ]:
function get_sample!(rnn::RNNwaveFn)
    for b in 1:rnn.Nb
        #rnn_state = zeros(Float32, rnn.N_hid)
        Flux.reset!(rnn.model)
        for i in 1:rnn.Ns
            prob = rnn.model(rnn.s0[i,:])
            rnn.sample[b, i] = rand(Categorical(prob), 1)
        end
    end
end
        

In [1]:
struct Ham
    Nst::Int
    Jz::Float64
    Bx::Float64
    Hl::Matrix{ComplexF64}
    HB::Matrix{ComplexF64}
end

function set_Ham(Jz::Float64, Bx::Float64)
    Nst::Int = 2
    Hl::Matrix{ComplexF64} = [Jz 0.0; 0.0 -Jz]
    HB::Matrix{ComplexF64} = [0.0 Bx; Bx 0.0]
    return Nst, Jz, Bx, Hl, HB
end

set_Ham (generic function with 1 method)

In [ ]:
function calc_local(H::Ham, R::RNNwaveFn)
    E_l = zeros(Float64, R.Nb)
    for s in 1:R.Ns-1
        for b in 1:R.Nb
            st1 = onehot(Int, 1:H.Nst, R.sample[b,s])
            st2 = onehot(Int, 1:H.Nst, R.sample[b,s+1])
            E_l[s] += real(s1' * H.Hl * s2)
            if(H.Bx != 0)
                for bp in b+1:R.Nb
                    st3 = onehot(Int, 1:H.Nst, R.sample[bp,s])
                    E_l[s] += real(s1' * H.HB * s3 + s3' * H.HB * s1)
                end
            end
        end       
    end
    return E_l
end

In [34]:
y = Multinomial(1, [0.1,0.1,0.8])
z = rand(y)
zz = onecold(z)

3

In [24]:
using StatsBase

In [33]:
using OneHotArrays

In [106]:
p = aweights([0.2, 0.8])
sample(1:2, p, 1)

1-element Vector{Int64}:
 2

In [109]:
A = Categorical([0.1999999, 0.8])
rand(A, 1)

LoadError: DomainError with [0.2, 0.8]:
Categorical: vector p is not a probability vector

In [74]:
logpdf(A,1)

-1.6094379124341003